In [1]:
import ollama
import joblib
import numpy as np
import pandas as pd
from tqdm import tqdm
from datasets import load_dataset
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

import os
import warnings
from utils import cv_trainer
from base import persian_text_preprocessing
warnings.filterwarnings('ignore')

In [ ]:
ds = load_dataset("hezarai/sentiment-dksf")
ds = ds.map(lambda example: {**example, 'text': persian_text_preprocessing(example['text'])}, 
           batched=False)
train_df: pd.DataFrame = ds["train"].to_pandas() # type: ignore
test_df: pd.DataFrame = ds["test"].to_pandas() # type: ignore
df = pd.concat([train_df, test_df])

# Dense Embeddings

In [6]:
def create_embeddings_batch(texts, model="nomic-embed-text", batch_size=300):
    all_embeddings = []
    texts_list = texts if isinstance(texts, list) else texts.to_list()
    
    for i in tqdm(range(0, len(texts_list), batch_size), desc="Creating embeddings"):
        batch = texts_list[i:i + batch_size]
        response = ollama.embed(model=model, input=batch)
        all_embeddings.extend(response["embeddings"])
    
    return np.array(all_embeddings)

embeddings_file = 'comments_embeddings.npy'

if os.path.exists(embeddings_file):
    print(f"Loading existing embeddings from {embeddings_file}")
    comments_dense_embeddings = np.load(embeddings_file)
else:
    print("Creating new embeddings...")
    comments_dense_embeddings = create_embeddings_batch(df["text"])
    
    np.save(embeddings_file, comments_dense_embeddings)
    print(f"Embeddings saved to {embeddings_file}")

print(f"Embeddings shape: {comments_dense_embeddings.shape}")

Creating new embeddings...


Creating embeddings: 100%|██████████| 104/104 [09:59<00:00,  5.77s/it]


Embeddings saved to comments_embeddings.npy
Embeddings shape: (30917, 768)


In [9]:
X_dense, y_dense = comments_dense_embeddings, df["label"].to_numpy()

train_size = len(train_df)
X_train_dense = X_dense[:train_size]
X_test_dense = X_dense[train_size:]
y_train_dense = y_dense[:train_size]
y_test_dense = y_dense[train_size:]

In [11]:
models_dense = {
     'logistic_regression': (
        Pipeline([
            ('scaler', StandardScaler()),
            ('pca', PCA()),
            ('classifier', LogisticRegression(random_state=42, max_iter=1000))
        ]),
        {
            'scaler': [None, StandardScaler()],
            'pca': [None, PCA(50), PCA(100), PCA(200)],
            'classifier__C': [0.1, 1.0, 10.0],
            'classifier__solver': ['liblinear', 'lbfgs'],
            'classifier__penalty': ['l1', 'l2'],
        }
    ),
    
    'gaussianNB': (
        Pipeline([
            ('scaler', StandardScaler()),
            ('pca', PCA()),
            ('classifier', GaussianNB())
        ]),
        {
            'scaler': [None, StandardScaler(), MinMaxScaler()],
            'pca': [None, PCA(50), PCA(100), PCA(200)],
            'classifier__var_smoothing': [1e-5, 1e-7, 1e-9]
        }
    ),
    
    'random_forest': (
        Pipeline([
            ('scaler', StandardScaler()),
            ('pca', PCA()),
            ('classifier', RandomForestClassifier(random_state=42))
        ]),
        {
            'scaler': [None, StandardScaler()],
            'pca': [None, PCA(100), PCA(200)],
            'classifier__n_estimators': [25, 75],
            'classifier__max_depth': [7, 12],
            'classifier__min_samples_split': [2, 5],
        }
    ),
    
    'mlp': (
        Pipeline([
            ('scaler', StandardScaler()),
            ('pca', PCA()),
            ('classifier', MLPClassifier(random_state=42, max_iter=500))
        ]),
        {
            'pca': [PCA(100), PCA(200)],
            'classifier__hidden_layer_sizes': [(100,), (100, 50), (200, 100)],
            'classifier__alpha': [0.0001, 0.001, 0.01],
            'classifier__learning_rate': ['constant', 'adaptive']
        }
    )
}

In [12]:
dense_results_output_path = 'dense_results.pkl'
if not os.path.exists(dense_results_output_path):
    print("Training dense embedding models...")
    dense_results = cv_trainer(
        X_train_dense, X_test_dense, y_train_dense, y_test_dense, 
        cv=3, models=models_dense # type: ignore
    )
    joblib.dump(dense_results, dense_results_output_path)
else:
    print("Loading dense results from file...")
    dense_results = joblib.load(dense_results_output_path)
    

Training dense embedding models...
Training logistic_regression...
Fitting 3 folds for each of 96 candidates, totalling 288 fits
[CV] END classifier__C=0.1, classifier__penalty=l1, classifier__solver=liblinear, pca=PCA(n_components=50), scaler=None; total time=   1.3s
[CV] END classifier__C=0.1, classifier__penalty=l1, classifier__solver=liblinear, pca=PCA(n_components=50), scaler=None; total time=   1.5s
[CV] END classifier__C=0.1, classifier__penalty=l1, classifier__solver=liblinear, pca=PCA(n_components=50), scaler=None; total time=   0.9s
[CV] END classifier__C=0.1, classifier__penalty=l1, classifier__solver=liblinear, pca=PCA(n_components=50), scaler=StandardScaler(); total time=   1.3s
[CV] END classifier__C=0.1, classifier__penalty=l1, classifier__solver=liblinear, pca=PCA(n_components=50), scaler=StandardScaler(); total time=   0.7s
[CV] END classifier__C=0.1, classifier__penalty=l1, classifier__solver=liblinear, pca=PCA(n_components=50), scaler=StandardScaler(); total time=   

In [13]:
dense_results = joblib.load(dense_results_output_path)
best_dense_model = dense_results['random_forest']['best_estimator']
print(f"best model: {dense_results['summary']['best_model_name']} params: {dense_results['random_forest']['best_params']}")
y_pred_dense = best_dense_model.predict(X_test_dense)
print(classification_report(y_test_dense, y_pred_dense))
print(confusion_matrix(y_test_dense, y_pred_dense))

best model: random_forest params: {'classifier__max_depth': 12, 'classifier__min_samples_split': 5, 'classifier__n_estimators': 75, 'pca': None, 'scaler': None}
              precision    recall  f1-score   support

           0       0.74      0.82      0.78      1107
           1       0.83      0.64      0.72      1032
           2       0.50      0.86      0.64       176

    accuracy                           0.74      2315
   macro avg       0.69      0.77      0.71      2315
weighted avg       0.76      0.74      0.74      2315

[[903 130  74]
 [300 656  76]
 [ 18   6 152]]


In [ ]:
digikala_scraped_comments_df = pd.read_csv('gemma3_4b_filtered_comments_labels_20250703_152906.csv', index_col=None)
digikala_scraped_comments_embeddings = create_embeddings_batch(digikala_scraped_comments_df['text'])
print("Test model performance on scraped digikala comments: ")
y_pred_digikala_dense = best_dense_model.predict(digikala_scraped_comments_embeddings)
print(classification_report(digikala_scraped_comments_df['label'].to_list(), y_pred_digikala_dense))
print(confusion_matrix(digikala_scraped_comments_df['label'].to_list(), y_pred_digikala_dense))

Creating embeddings: 100%|██████████| 9/9 [00:50<00:00,  5.56s/it]

Test model performance on scraped digikala comments: 
              precision    recall  f1-score   support

           0       0.42      0.61      0.50       616
           1       0.86      0.59      0.70      1791
           2       0.06      0.25      0.10        88

    accuracy                           0.59      2495
   macro avg       0.45      0.48      0.43      2495
weighted avg       0.72      0.59      0.63      2495

[[ 377  139  100]
 [ 491 1061  239]
 [  29   37   22]]


# Discrete Embeddings

In [7]:
train_size = len(train_df)
X_discrete, y_discrete = df["text"].to_numpy(), df["label"].to_numpy()
X_train_discrete, X_test_discrete, y_train_discrete, y_test_discrete = X_discrete[:train_size], X_discrete[train_size:], y_discrete[:train_size], y_discrete[train_size:]

In [8]:
persian_stop_words = [
    # Articles and determiners
    'این', 'آن', 'یک', 'هر', 'همه', 'تمام', 'کل', 'چند', 'بعض', 'برخی',
    
    # Pronouns
    'من', 'تو', 'او', 'ما', 'شما', 'آنها', 'خود', 'خودم', 'خودت', 'خودش',
    
    # Prepositions (neutral ones)
    'در', 'به', 'از', 'با', 'تا', 'روی', 'زیر', 'کنار', 'داخل', 'خارج',
    
    # Simple conjunctions (not contrastive)
    'و', 'یا', 'که', 'چون', 'وقتی', 'زمانی', 'هنگامی',
    
    # Neutral verbs
    'می', 'شود', 'کند', 'دهد', 'است', 'باشد' ,'است', 'بود', 'شد', 'کرد', 'داشت', 'دارد', 'خواهد', 'باید', 'می‌شود',
    
    # Object markers and particles
    'را', 'رو', 'های', 'ها', 'ان', 'ات', 'تان', 'شان',
    
    # Time references (neutral)
    'امروز', 'دیروز', 'فردا', 'حالا', 'الان', 'وقت', 'زمان',
    
    # Place references
    'اینجا', 'آنجا', 'کجا', 'جا', 'محل', 'مکان',
    
    # Common neutral words
    'چیز', 'کار', 'راه', 'نوع', 'قسم', 'طور', 'مثل', 'مانند', 'نام', 'اسم'
]

models_discrete = {
    'logistic_tfidf': (
        Pipeline([
            ('vectorizer', TfidfVectorizer()), 
            ('scaler', StandardScaler()),
            ('pca', PCA()),
            ('logisticClassifier', LogisticRegression())
        ]),
        {
            'vectorizer__stop_words': [None, persian_stop_words],
            'vectorizer__max_features': [500, 2000, None],
            'scaler': [None, StandardScaler(with_mean=False)],
            'pca': [None, PCA(50), PCA(100)],
            'logisticClassifier__C': [0.01, 0.1, 1, 10],
            'logisticClassifier__penalty': ['l2'],
            'logisticClassifier__max_iter': [1000], 
        }
    ),
    
    'naive_bayes_count': (
        Pipeline([
            ('vectorizer', CountVectorizer()),
            ('classifier', MultinomialNB())
        ]),
        {
            'vectorizer__stop_words': [None, persian_stop_words],
            'vectorizer__max_features': [500, 1000, 2000],
            'vectorizer__ngram_range': [(1, 1), (1, 2)],
            'vectorizer__min_df': [1, 2],
            'classifier__alpha': [0.1, 0.5, 1.0, 2.0]
        }
    ),
    
    'random_forest_tfidf': (
        Pipeline([
            ('vectorizer', TfidfVectorizer()),
            ('classifier', RandomForestClassifier(random_state=42))
        ]),
        {
            'vectorizer__stop_words': [None, persian_stop_words],
            'vectorizer__max_features': [1000, 2000],
            'vectorizer__ngram_range': [(1, 1), (1, 2)],
            'classifier__n_estimators': [100, 200],
            'classifier__max_depth': [10, 20, None],
        }
    ),
}

In [9]:
discrete_results_output_path = 'discrete_results.pkl'
if not os.path.exists(discrete_results_output_path):
    print("Training discrete embedding models...")
    discrete_results = cv_trainer(
        X_train_discrete, X_test_discrete, y_train_discrete, y_test_discrete, 
        cv=3, models=models_discrete # type: ignore
    )
    joblib.dump(discrete_results, discrete_results_output_path)
else:
    discrete_results = joblib.load(discrete_results_output_path)
    print("Loaded discrete results from file")

Loaded discrete results from file


In [13]:
best_model_name = discrete_results['summary']['best_model_name']
best_discrete_model = discrete_results[best_model_name]['best_estimator']
print(f"best model: {discrete_results['summary']['best_model_name']} params: {discrete_results['random_forest_tfidf']['best_params']}")
y_pred_discrete = best_discrete_model.predict(X_test_discrete)
print(classification_report(y_test_discrete, y_pred_discrete))
print(confusion_matrix(y_test_discrete, y_pred_discrete))

best model: random_forest_tfidf params: {'classifier__max_depth': None, 'classifier__n_estimators': 200, 'vectorizer__max_features': 2000, 'vectorizer__ngram_range': (1, 2), 'vectorizer__stop_words': None}
              precision    recall  f1-score   support

           0       0.88      0.88      0.88      1107
           1       0.93      0.79      0.86      1032
           2       0.51      0.98      0.68       176

    accuracy                           0.85      2315
   macro avg       0.78      0.88      0.80      2315
weighted avg       0.88      0.85      0.85      2315

[[975  57  75]
 [128 816  88]
 [  2   1 173]]


In [18]:
feature_names = best_discrete_model.named_steps['vectorizer'].get_feature_names_out()
feature_importances = best_discrete_model.named_steps['classifier'].feature_importances_

feature_importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': feature_importances
}).sort_values('importance', ascending=False)

print(feature_importance_df.head(10))

       feature  importance
1151      عالی    0.026685
115       اصلا    0.019080
428        بود    0.018924
1154     عالیه    0.013610
685        خوب    0.010724
724       خیلی    0.009948
1681       ولی    0.009054
721     خوشمزه    0.007562
689       خوبه    0.007491
1152  عالی بود    0.007170


In [ ]:
digikala_scraped_comments_df = pd.read_csv('gemma3_4b_filtered_comments_labels_20250703_152906.csv', index_col=None)
print("Test model performance on scraped digikala comments: ")
y_pred_digikala_dense = best_discrete_model.predict(digikala_scraped_comments_df['text'])
print(classification_report(digikala_scraped_comments_df['label'].to_list(), y_pred_digikala_dense))
print(confusion_matrix(digikala_scraped_comments_df['label'].to_list(), y_pred_digikala_dense))

Test model performance on scraped digikala comments: 
              precision    recall  f1-score   support

           0       0.67      0.52      0.59       616
           1       0.91      0.74      0.82      1791
           2       0.07      0.45      0.12        88

    accuracy                           0.68      2495
   macro avg       0.55      0.57      0.51      2495
weighted avg       0.82      0.68      0.74      2495

[[ 323   92  201]
 [ 143 1326  322]
 [  15   33   40]]
